In [ ]:
from langdetect import detect
import os
import re
from collections import defaultdict

In [ ]:
import pandas as pd

data = defaultdict(lambda: {'topic': '', 'content': '', 'messages': []})

for file_path in os.listdir("cleaned_intercom_messages"):
    csv = pd.read_csv(f"cleaned_intercom_messages/{file_path}")
    csv = csv[csv['clean_body'].notna()]
    bodies = csv["clean_body"]
    ids = zip(csv["key_intercomconversation"], csv["key_iteration"])
    for id, body in zip(ids, bodies):
        m = re.search(" Selected category: (.+) Content: (.+) uid:", body)
        if m is not None:
            data[id]['topic'] = m.group(1)
            data[id]['content'] = m.group(2)
        else:
            data[id]['messages'].append(body)

df = pd.DataFrame.from_dict(data, orient="index")
print(len(df[df["topic"] == '']))
print(len(df[df["topic"] != '']))

In [ ]:
import re

In [ ]:
messages = {}

#for index, row in test_df.iterrows():
for index, row in df.iterrows():
    msg = re.sub(r'{{[^}]*}}', '', " ".join(row['messages']))
    messages[index] = msg

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

In [ ]:
stop_words.extend(["wo", "n't", "'m", "ca", "'ll", "'re", "'ve", "'d"])

In [ ]:
#words = word_tokenize(messages[13850823877])
#words = messages[13850823877].split(" ")
lemmatizer = WordNetLemmatizer()

lemmas = {}
stop_words = set(stop_words)
length = len(messages)
for i, (key, value) in enumerate(messages.items()):
    words = word_tokenize(value)
    lem = []
    if i % (length // 100) == 0:
        print(i, length)
    for word in words:
        lemma = lemmatizer.lemmatize(word)
        if len(lemma) > 1:
            lem.append(lemma.lower())
    lem = [w for w in lem if not w in stop_words]
    lemmas[key] = lem
    
#print(lemmas)

In [ ]:
all_lems = []

for key in lemmas:
    all_lems.extend(lemmas[key])

In [ ]:
from collections import Counter

counter = Counter(all_lems).most_common()

In [ ]:
stop_words_from_freq = ["pipedrive", "please", "help", "know", "let", "see", "'s", "need", "would", "question", "``", "day", "get", "one", "possible", "feel", "wa", "''", "ha", "want", "like", "thank", "already", "yes", "ok", "done", "still", "great", "could", "soon", "also", "anything", "well", "right", "sure", "um", "else"]

In [ ]:
stop_words_from_freq = set(stop_words_from_freq)

In [ ]:
new_lemmas = dict()

for key in lemmas:
    new_lemmas["_".join(map(str, key))] = [word for word in lemmas[key] if word not in stop_words_from_freq]

In [ ]:
import json

with open("new_lems.json", "w", encoding="UTF-8") as f:
    f.write(json.dumps(new_lemmas))